In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import re
import sys

import skimage.io as skio
import matplotlib.pyplot as plt

import numpy as np
import os
import sys
from math import sqrt

import cv2
import pandas as pd
import albumentations as A

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
from sklearn.model_selection import train_test_split
from scipy.special import expit, logit
from skimage.transform import rescale, resize, downscale_local_mean

import time

from PIL import Image
from skimage.feature import hog

from sklearn.metrics import confusion_matrix
import itertools

import tensorflow as tf
from tensorflow import keras
from keras.utils import generic_utils
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D , BatchNormalization
from keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adadelta,Adam

from sklearn.metrics import classification_report

import seaborn as sns

In [3]:
def ReadImageFolder(path):
    filepaths = [os.path.join(r,file) for r,d,f in os.walk( path) for file in f]
    ext = (".tif",".TIF",".jpg")
    filepaths = [x for x in filepaths if x.endswith(ext)]
    return filepaths

In [4]:
Labels = [ ]
Labels_String = [ "Motorbikes" ,"helicopter","airplanes","brain", "car_side","panda",
                 "chandelier", "cougar_face","crab","grand_piano"]

path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/Motorbikes"
Motorbikes = ReadImageFolder(path)
for i in range(len(Motorbikes)):
  Labels.append(0)
path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/helicopter"
helicopter = ReadImageFolder(path)
for i in range(len(helicopter)):
  Labels.append(1)

path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/airplanes"
airplanes = ReadImageFolder(path)
for i in range(len(airplanes)):
  Labels.append(2)

path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/brain"
brain = ReadImageFolder(path)
for i in range(len(brain)):
  Labels.append(3)

path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/car_side"
car_side = ReadImageFolder(path)
for i in range(len(car_side)):
  Labels.append(4)

path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/panda"
panda = ReadImageFolder(path)
for i in range(len(panda)):
  Labels.append(5)

path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/chandelier"
chandelier = ReadImageFolder(path)
for i in range(len(chandelier)):
  Labels.append(6)


path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/cougar_face"
cougar_face = ReadImageFolder(path)
for i in range(len(cougar_face)):
  Labels.append(7)

path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/crab"
crab = ReadImageFolder(path)
for i in range(len(crab)):
  Labels.append(8)

path = "/content/drive/MyDrive/ColabNotebooks/Dataset2Pattern/grand_piano"
grand_piano = ReadImageFolder(path)
for i in range(len(grand_piano)):
  Labels.append(9)


Paths_Array= [ Motorbikes ,helicopter,airplanes,brain, car_side,panda, chandelier, cougar_face,crab,grand_piano]

print(Paths_Array)

[['/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_001.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_002.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_072.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_067.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_070.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_009.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_007.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_068.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_011.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_004.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_069.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_010.tif', '/content/drive/MyDrive/ColabNotebooks/DatasetPattern/actin/actin_071.tif', '/content/

In [5]:
def PrepareImagesArray(paths,AllImages_Array):

    for i in paths:
     
      img_arr = cv2.imread(i)[...,::-1] #convert BGR to RGB format
      img_size = 256
      img_arr = img_arr.astype(np.float64)
    # Reshaping images to preferred size
      img_arr = cv2.resize(img_arr, (img_size, img_size) , interpolation = cv2.INTER_CUBIC  ) 
      img_arr = (img_arr - img_arr.mean()) / (img_arr.std()) 
    
      AllImages_Array.append(img_arr)

In [6]:
AllImages_Array = [ ]
for i in Paths_Array:
  PrepareImagesArray(i,AllImages_Array)
AllImages_Array_np = np.array(AllImages_Array)

# Delete old images array
AllImages_Array = [ ]


In [7]:
Labels_encoded = to_categorical(Labels, num_classes = 10)

In [11]:
x_train , x_test , y_train , y_test = train_test_split(AllImages_Array_np , Labels_encoded, test_size=0.15)
x_train , x_valid , y_train , y_valid = train_test_split(x_train , y_train, test_size=0.18)
print("x_train shape",x_train.shape)
print("x_test shape",x_test.shape)

print("x_valid shape",x_valid.shape)
print("y_train shape",y_train.shape)
print("y_test shape",y_test.shape)
print("y_valid shape",y_valid.shape)

# Delete images after splitting to save memory

AllImages_Array_np = [ ]

x_train shape (606, 382, 382, 3)
x_test shape (131, 382, 382, 3)
x_valid shape (134, 382, 382, 3)
y_train shape (606, 10)
y_test shape (131, 10)
y_valid shape (134, 10)


In [27]:
input_t = keras.Input(shape=(256,256,3))
res_model = keras.applications.ResNet50(include_top = False , weights="imagenet",input_tensor=input_t)

In [28]:
for layer in res_model.layers[:143]:
  layer.trainable = False

In [29]:
to_res = (256,256)
model = keras.models.Sequential()
model.add(keras.layers.Lambda(lambda image: tf.image.resize(image,to_res)))
model.add(res_model)
model.add(keras.layers.Flatten())
model.add(BatchNormalization())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(keras.layers.Dense(10, activation='softmax'))

In [30]:
check_point = keras.callbacks.ModelCheckpoint(filepath="resnet_test" , monitor='val_accuracy',mode="max",save_best_only = True)



In [ ]:
model.compile( loss = 'categorical_crossentropy' , optimizer = "Adam",
              metrics = ['accuracy'])

history = model.fit(x_train,y_train,batch_size = 16 , epochs = 10, verbose = 1 , 
                    validation_data = (x_valid,y_valid), callbacks =[check_point])

model.summary()

In [ ]:
test_eval = model.evaluate(x_test, y_test, verbose=1)

In [43]:
model.load_weights("resnet_test")
test_eval = model.evaluate(x_test, y_test, verbose=1)

5/5 [==============================] - 3s 451ms/step - loss: 2.4879 - accuracy: 0.7939


In [ ]:
model.save('Resnet_ImagesColored.h5py')

In [ ]:
!zip -r /content/Resnet_ImagesColored.zip /content/Resnet_ImagesColored.h5py

In [38]:
model_2 = keras.models.load_model("Resnet_ImagesColored.h5py")

In [45]:
predicted_classes_all = model.predict(x_test)
predicted_classes_all = np.argmax(predicted_classes_all,axis=1)

In [46]:
True_classes_all = np.argmax(y_test,axis = 1) 

In [47]:
target_names = ["Class: {}".format(Labels_String[i]) for i in range(10)]
print(classification_report(True_classes_all, predicted_classes_all, target_names=target_names))

                     precision    recall  f1-score   support

       Class: actin       0.93      0.88      0.90        16
         Class: dna       0.92      1.00      0.96        11
    Class: endosome       0.71      0.67      0.69        15
          Class: er       0.65      1.00      0.79        13
      Class: golgia       0.67      0.92      0.77        13
      Class: golgpp       1.00      0.53      0.70        15
    Class: lysosome       0.82      0.60      0.69        15
Class: microtubules       0.71      0.62      0.67         8
Class: mitochondria       1.00      0.73      0.84        11
   Class: nucleolus       0.78      1.00      0.88        14

           accuracy                           0.79       131
          macro avg       0.82      0.80      0.79       131
       weighted avg       0.82      0.79      0.79       131



In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()